In [1]:
import gc
import numpy as np
import pandas as pd
from utils.logging_setup import init_logging, get_logger

init_logging(level="INFO")  # "INFO" o "DEBUG"
log = get_logger("notebook")

log.info("Notebook listo.")
log.debug("Mensaje solo visible si level es DEBUG.")

from utils.OpenDataTools import OpenDataTools

[notebook] - [INFO] - Notebook listo.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Se inicializa el buscador de datos abiertos
open_data_tools = OpenDataTools()

[root] - [WARNING] - Requests made without an app_token will be subject to strict throttling limits.


In [3]:
# Los headers están en la segunda fila del archivo
matriz = pd.read_excel('/app/data/20260204_Plantilla_Matriz_de_Contratacion - copia.xlsx', sheet_name='Matriz', header=1)

In [4]:
matriz.columns

Index(['REGIONAL', 'CENTRO ZONAL', 'MUNICIPIO', 'Componente para la UDS',
       'SERVICIO SIM', 'NOMBRE DEL SERVICIO', 'ZONA 2026.', 'ATENCIONES',
       'UDS_Zona', 'Cupos_Zona', 'UDS', 'Meses', 'Forma de Contratación',
       'NIT CONTRATISTA 2026', 'CONTRATISTA 2026',
       'Es Asociacion de Padres y Madres Usuarias del Servicio', 'Fecha_FIn',
       'Valor 2025', 'Valor 2026', 'Aporte ICBF',
       'VALOR INICIAL OTROS CONCEPTOS\n$',
       'CANTIDAD ATENCIONES QUE \nAPLICA TASAS COMPENSATORIAS\n(HI)',
       'VR. TASAS COMPENSATORIAS (SOLO APLICA PARA HOGARES INFANTILES )\n$',
       'VALOR TOTAL INICIAL APORTE ICBF POR SERVICIO\n$',
       'VALOR TOTAL INICIAL APORTE ICBF POR CONTRATO (REGISTRO UNICO)\n$',
       '% APORTE EAS CONTRAPARTIDA', 'VALOR  CONTRAPARTIDA APORTE EAS\n$',
       'VALOR TOTAL INICAL DEL APORTE DE LA EAS (REGISTRO UNICO)\n$',
       'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$',
       'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO) EN SMLV',
       'ALERTA V

In [5]:
nits = matriz[~matriz['NIT CONTRATISTA 2026'].isna()]['NIT CONTRATISTA 2026'].unique()
names_workers = matriz[~matriz['CONTRATISTA 2026'].isna()]['CONTRATISTA 2026'].unique()

print('nits:', len(nits))
print('names_workers:', len(names_workers))

nits: 1350
names_workers: 1276


In [6]:
def get_order_identifiers_as_list(order_identificators_list: list, chunk_size: int = 1000):
    chunks = [order_identificators_list[i:i + chunk_size] for i in range(0, len(order_identificators_list), chunk_size)]
    return ["(" + ", ".join([f'"{elem}"' for elem in chunk]) + ")" for chunk in chunks]

In [7]:
nits_as_str = get_order_identifiers_as_list(nits, chunk_size=100)

In [8]:
contracts_by_nit = []
for nits_chain in nits_as_str:
    contracts_by_nit.append(open_data_tools.get_contratos_por_cedulas(nits_chain, limit=1000000))

Se identificaron 539 registros en SECOP I.
Se identificaron 1745 registros en SECOP II.
Se identificaron 594 registros en SECOP I.
Se identificaron 1620 registros en SECOP II.
Se identificaron 1249 registros en SECOP I.
Se identificaron 3112 registros en SECOP II.
Se identificaron 436 registros en SECOP I.
Se identificaron 1170 registros en SECOP II.
Se identificaron 1033 registros en SECOP I.
Se identificaron 2016 registros en SECOP II.
Se identificaron 623 registros en SECOP I.
Se identificaron 1244 registros en SECOP II.
Se identificaron 328 registros en SECOP I.
Se identificaron 967 registros en SECOP II.
Se identificaron 496 registros en SECOP I.
Se identificaron 1420 registros en SECOP II.
Se identificaron 586 registros en SECOP I.
Se identificaron 1526 registros en SECOP II.
Se identificaron 692 registros en SECOP I.
Se identificaron 1427 registros en SECOP II.
Se identificaron 810 registros en SECOP I.
Se identificaron 1141 registros en SECOP II.
Se identificaron 735 registros 

In [9]:
secopi = []
secopii = []

for group_result in contracts_by_nit:
    # Specific filters:
    # SECOP I
    group_result['SECOP_I']['anno_cargue_secop'] = group_result['SECOP_I']['anno_cargue_secop'].astype(int)
    group_result['SECOP_I'] = group_result['SECOP_I'][group_result['SECOP_I']['nit_de_la_entidad'].str.contains('89999923')]
    secopi.append(group_result['SECOP_I'][group_result['SECOP_I']['anno_cargue_secop'] >= 2024])
    # SECOP II
    group_result['SECOP_II'] = group_result['SECOP_II'][group_result['SECOP_II']['nit_entidad'].str.contains('89999923')]
    secopii.append(group_result['SECOP_II'][group_result['SECOP_II']['fecha_de_firma'] >= '2024-01-01'])

secopi = pd.concat(secopi)
secopii = pd.concat(secopii)

In [10]:
print('matriz:', matriz.shape)
print('secopi:', secopi.shape)
print('secopii:', secopii.shape)

matriz: (4187, 34)
secopi: (3, 26)
secopii: (7620, 18)


In [11]:
matriz['NIT CONTRATISTA 2026'] = matriz['NIT CONTRATISTA 2026'].astype(str)

In [12]:
secopii.columns

Index(['id_contrato', 'fecha_de_firma', 'tipo_de_contrato',
       'modalidad_de_contratacion', 'estado_contrato', 'objeto_del_contrato',
       'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
       'tipo_de_identificaci_n_representante_legal',
       'identificaci_n_representante_legal', 'nombre_representante_legal',
       'nombre_entidad', 'nit_entidad', 'departamento', 'ciudad',
       'valor_del_contrato', 'urlproceso'],
      dtype='object')

In [25]:
from typing import List, Tuple

def fuzzy_merge(merge_column_pairs: List[List[str]], pct: float, matriz: pd.DataFrame, secop: pd.DataFrame) -> pd.DataFrame:
    # merge_column_pairs: [valor_l, valor_r, texto_l, texto_r]]
    merge_result = []
    for pair in merge_column_pairs:
        valor_l = pair[0]; valor_r = pair[1]
        texto_l = pair[2]; texto_r = pair[3]
        
        # --- 0) ids para controlar 1-1 ---
        L = matriz.copy()
        R = secop.copy()

        L[valor_l] = L[valor_l].astype(float)
        R[valor_r] = R[valor_r].astype(float)

        L["_id_l"] = np.arange(len(L))
        R["_id_r"] = np.arange(len(R))

        # --- 1) números ---
        L[valor_l] = pd.to_numeric(L[valor_l], errors="coerce")
        R[valor_r] = pd.to_numeric(R[valor_r], errors="coerce")

        L = L.dropna(subset=[valor_l]).sort_values(valor_l).copy()
        R = R.dropna(subset=[valor_r]).sort_values(valor_r).copy()

        # --- 2) candidato por cercanía ---
        cand = pd.merge_asof(
            L,
            R,
            left_on=valor_l,
            right_on=valor_r,
            direction="nearest"
        )

        # --- 3) texto: normaliza a dígitos (NIT/documento) ---
        def only_digits(x):
            if pd.isna(x):
                return ""
            return "".join(ch for ch in str(x) if ch.isdigit())

        l_doc = cand[texto_l].map(only_digits)
        r_doc = cand[texto_r].map(only_digits)

        txt_ok = (l_doc == r_doc) & (r_doc != "")

        # --- 4) tolerancia porcentual ---
        den = cand[valor_l].abs()
        diff = (cand[valor_l] - cand[valor_r]).abs()
        tol_ok = np.where(den.eq(0), diff.eq(0), diff <= (pct * den))

        cand = cand[txt_ok & tol_ok].copy()

        # --- 5) score: menor diferencia porcentual = mejor ---
        cand["diff_abs"] = diff[txt_ok & tol_ok]
        cand["diff_pct"] = np.where(den[txt_ok & tol_ok].eq(0), 0, cand["diff_abs"] / den[txt_ok & tol_ok])

        # --- 6) resolver colisiones para que sea 1-1 ---
        # Paso A: cada L se queda con su mejor R
        cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

        # Paso B: si varios L apuntan al mismo R, nos quedamos con el mejor (y descartamos el resto)
        cand = cand.sort_values(["_id_r", "diff_pct"]).drop_duplicates("_id_r", keep="first")

        # (Opcional) Paso C: volver a asegurar unicidad por L (por si el paso B dejó huecos)
        cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

        out = cand.drop(columns=["diff_abs", "diff_pct", "_id_r", "_id_l"])

        merge_result.append(out)
    
    return pd.concat(merge_result)

In [26]:
pct = 0.4  # 3% tolerancia

merge_column_pairs = [
    ['VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$', 'valor_del_contrato', 'NIT CONTRATISTA 2026', 'documento_proveedor'],   
    ['VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$', 'valor_del_contrato', 'NIT CONTRATISTA 2026', 'identificaci_n_representante_legal'],   
]

merge = fuzzy_merge(merge_column_pairs, pct, matriz, secopii)

In [27]:
merge.shape

(859, 52)

In [28]:
merge.to_excel('/app/data/cruce_secop_matriz.xlsx', index=False)

In [31]:
matriz[matriz['NIT CONTRATISTA 2026'] == '900937582'][[
    'VALOR INICIAL OTROS CONCEPTOS\n$', 
    'VALOR TOTAL INICIAL APORTE ICBF POR SERVICIO\n$',
    'VALOR TOTAL INICIAL APORTE ICBF POR CONTRATO (REGISTRO UNICO)\n$',
    'VALOR TOTAL INICAL DEL APORTE DE LA EAS (REGISTRO UNICO)\n$',
    'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$',
    'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO) EN SMLV'
    ]]

,VALOR INICIAL OTROS CONCEPTOS\n$,VALOR TOTAL INICIAL APORTE ICBF POR SERVICIO\n$,VALOR TOTAL INICIAL APORTE ICBF POR CONTRATO (REGISTRO UNICO)\n$,VALOR TOTAL INICAL DEL APORTE DE LA EAS (REGISTRO UNICO)\n$,VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$,VALOR TOTAL DEL CONTRATO (UNICO REGISTRO) EN SMLV
0,NaN,1.052984e+09,1.052984e+09,2.105968e+07,1.074044e+09,754.509211
1,NaN,4.380543e+08,6.647591e+08,1.329518e+07,6.780543e+08,476.328980
2,NaN,2.267048e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
